In [1]:
import sys
sys.executable

'/mnt/c/users/nawfal/documents/apps/collaborative project/cctv_motion_detection/motion_venv/bin/python'

In [2]:
import cv2
import os
import time
import numpy as np
import csv

# Camera Check

In [39]:
# cap = cv2.VideoCapture("http://192.168.50.234:5000/video")
cap = cv2.VideoCapture("../test/vidio/record_1.mp4")


In [37]:
# Initialize the camera

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

# Capture frames from the camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Display the frame
    cv2.namedWindow("Camera Capture", cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow("Camera Capture", 640, 480)
    cv2.imshow("Camera Capture", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(22) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Camera is ready. Press 'q' to quit.


# Import Mediapipe

In [3]:
import cv2
import os
import time
import csv
import warnings
import pickle

import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


2025-09-04 16:42:36.754038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756975356.988810     868 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756975357.058363     868 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756975357.666045     868 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756975357.666087     868 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756975357.666089     868 computation_placer.cc:177] computation placer alr

# Z Coordinate detection
Zoom in & Zoom Out while move forward and backwards

In [4]:
def check_nose_z(lndmrk_z):
    nose = lndmrk_z["nose"]
    ear_l = lndmrk_z["ear_l"]
    ear_r = lndmrk_z["ear_r"]

    if nose < (ear_l and ear_r):
        return "Hadap Depan"
    else:
        return "Hadap Belakang"

    return f"nose: {nose:.5f}, ear: {ear_l:.5f}"

In [ ]:
lndmrkX = None
lndmrkY = None
lndmrkZ = None

# cap = cv2.VideoCapture("http://192.168.100.197:5000/video")
cap = cv2.VideoCapture("../test/vidio/record_1.mp4")


time.sleep(3)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            # Get specific landmarks
            nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
            ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
            ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]

            show_landmark_list = landmark_pb2.NormalizedLandmarkList()
            show_landmark_list.landmark.extend([nose])

            nose_z = check_nose_z({"nose": nose.z, "ear_l": ear_l.z, "ear_r": ear_r.z})

            # Draw landmarks
            for lndmrk_z in show_landmark_list.landmark:
                x, y = int(lndmrk_z.x * image.shape[1]), int(lndmrk_z.y * image.shape[0])
                lndmrkX = lndmrk_z.x
                lndmrkY = lndmrk_z.y
                lndmrkZ = lndmrk_z.z

            cv2.putText(
                image,
                f"x: ({nose_z})",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )

            cv2.imshow('Raw Webcam Feed', image)

        except:
            cv2.putText(
                image,
                f"x: (No Detected)",
                (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 0),
                2,
                cv2.LINE_AA
            )

            cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(12) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1756882411.452240  116717 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1756882411.486454  119074 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1756882411.899384  119064 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756882412.072719  119066 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756882412.086042  119064 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756882412.087901  119071 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

move forward goes till minus

# Face Direction Detection

In [21]:
def check_direction(landmark):
    nose = landmark["nose"]
    ear_r = landmark["ear_r"]
    ear_l = landmark["ear_l"]

    smallest_value = min(landmark, key=landmark.get)
    highest_value = max(landmark, key=landmark.get)
    
    if smallest_value == "nose":
        direction = "Right"
    elif highest_value == "nose":
        direction = "Left"
    else:
        direction = "Center"

    return direction


# Camera Check

In [22]:
# camera = cv2.VideoCapture("http://192.168.100.197:5000/video")
camera = cv2.VideoCapture("../test/vidio/record_1.mp4")


# Check if the camera opened successfully
if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.1, min_tracking_confidence=0.1) as holistic :
    
    while camera.isOpened():  
        ret, frame = camera.read()

        if not ret:
            print("❌ Failed to grab frame")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # =============== THE LANDMARKS ============== #
        try:
            nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
            ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
            ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
            wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

            # Array Each Landmark
            face_direction = check_direction({"nose": nose.x, "ear_r": ear_r.x, "ear_l": ear_l.x}) 

            # For Draw
            lm = landmark_pb2.NormalizedLandmarkList()
            lm.landmark.extend([nose, ear_r, ear_l, wrist_r])

            # ============================================ #

            for lndmrk_z in lm.landmark:
                x, y = int(lndmrk_z.x * image.shape[1]), int(lndmrk_z.y * image.shape[0])
                cv2.circle(image, (x, y), 5, (255, 0, 0), -1)
            
                cv2.putText(
                    image,
                    f"x: ({face_direction})",
                    (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2,
                    cv2.LINE_AA
                )
            # =============== DETECTION ================== #
            # ============================================ #

            cv2.imshow('Raw Webcam Feed', image)
        except Exception as e:
            

            cv2.putText(
                    image,
                    f"x: (Error {e})",
                    (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2,
                    cv2.LINE_AA
                )
            # =============== DETECTION ================== #
            # ============================================ #

            cv2.imshow('Raw Webcam Feed', image)


        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

Camera is ready. Press 'q' to quit.


I0000 00:00:1756883084.383355  116717 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1756883084.415026  121328 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1756883084.898511  121318 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756883084.972209  121323 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756883084.986803  121316 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756883084.988027  121320 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

Left & Right Eye Can Be Used For the Left and Right

# Body Rotation State

### Getting State Of Body Direction Facing

In [30]:
def side_state(state, lndmrkX):
    if state == 0:
        if lndmrkX["nose"] == min(lndmrkX.values()):
            direction = "Kanan"
        elif lndmrkX["nose"] == max(lndmrkX.values()):      
            direction = "Kiri"
        else:
            direction = "Tengah"
    else:
        if lndmrkX["nose"] == min(lndmrkX.values()):
            direction = "Kiri"
        elif lndmrkX["nose"] == max(lndmrkX.values()):      
            direction = "Kanan"
        else:
            direction = "Tengah"
    
    return direction

def hand_state(state, lndmrkZ):

    smallest_value = min(lndmrkZ, key=lndmrkZ.get)
    highest_value = max(lndmrkZ, key=lndmrkZ.get)

    hand = smallest_value
    wristR = f"{lndmrkZ['wrist_r']:.2f}"
    nose = f"{lndmrkZ['nose']:.2f}"
    
    # return f"WristR: {wristR} | Nose: {nose}"

    if state == 0:
        if lndmrkZ["wrist_r"] < lndmrkZ["nose"]:
            hand = "Terlihat"
        else:
            hand = "Tidak Terlihat"
    else:
        if lndmrkZ["wrist_r"] > lndmrkZ["nose"]:
            hand = "Terlihat"
        else:
            hand = "Tidak Terlihat"
    return hand

def get_direction_state(lndmrk):
    noseX, noseY, noseZ = lndmrk["nose"].x, lndmrk["nose"].y, lndmrk["nose"].z
    earLX, earLY, earLZ = lndmrk["ear_l"].x, lndmrk["ear_l"].y, lndmrk["ear_l"].z
    earRX, earRY, earRZ = lndmrk["ear_r"].x, lndmrk["ear_r"].y, lndmrk["ear_r"].z
    wristRX, wristRY, wristRZ = lndmrk["wrist_r"].x, lndmrk["wrist_r"].y, lndmrk["wrist_r"].z
    wristLX, wristLY, wristLZ = lndmrk["wrist_l"].x, lndmrk["wrist_l"].y, lndmrk["wrist_l"].z
    
    lndmrkX = {"nose": noseX, "ear_l": earLX, "ear_r": earRX, "wrist_r": wristRX, "wrist_l": wristLX }
    lndmrkY = {"nose": noseY, "ear_l": earLY, "ear_r": earRY, "wrist_r": wristRY, "wrist_l": wristLY }
    lndmrkZ = {"nose": noseZ, "ear_l": earLZ, "ear_r": earRZ, "wrist_r": wristRZ, "wrist_l": wristLZ }

    if noseZ < (earLZ and earRZ):
        state = "Hadap Depan"
        side = side_state(0, lndmrkX)
        handZ = hand_state(0, lndmrkZ)
    else:
        state = "Hadap Belakang"
        side = side_state(1, lndmrkX)
        handZ = hand_state(1, lndmrkZ)

    return f"Tangan Z: {handZ}"
    return f"{state} || Ke {side} || Tangan Z: {handZ}"
    # return f"Z : {handZ}"



In [36]:
# camera = cv2.VideoCapture("http://192.168.100.197:5000/video")
camera = cv2.VideoCapture("../test/vidio/record_1.mp4")

# Check if the camera opened successfully
if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.1) as holistic :
    
    while camera.isOpened():  
        ret, frame = camera.read()

        if not ret:
            print("❌ Failed to grab frame")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # =============== THE LANDMARKS ============== #
        try:
            nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
            ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
            ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
            wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
            wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_WRIST]

            # Array Each Landmark
            state_direction = get_direction_state({"nose": nose, "ear_l": ear_l, "ear_r": ear_r, "wrist_r": wrist_r, "wrist_l": wrist_l})

            # For Draw
            lm = landmark_pb2.NormalizedLandmarkList()
            lm.landmark.extend([nose, ear_r, ear_l, wrist_r])

            # ============================================ #

            for lndmrk_z in lm.landmark:
                x, y = int(lndmrk_z.x * image.shape[1]), int(lndmrk_z.y * image.shape[0])
                cv2.circle(image, (x, y), 5, (255, 0, 0), -1)
            
                cv2.putText(
                    image,
                    f"x: ({state_direction})",
                    (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2,
                    cv2.LINE_AA
                )
            # =============== DETECTION ================== #
            # ============================================ #

            cv2.imshow('Raw Webcam Feed', image)
        except Exception as e:
            

            cv2.putText(
                    image,
                    f"x: (Error {e})",
                    (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2,
                    cv2.LINE_AA
                )
            
            # =============== DETECTION ================== #
            # ============================================ #

            cv2.imshow('Raw Webcam Feed', image)


        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

Camera is ready. Press 'q' to quit.


I0000 00:00:1756976647.946224     868 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1756976647.975640   40756 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1756976648.281368   40744 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756976648.497435   40745 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756976648.513253   40746 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756976648.513814   40743 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00